<a href="https://colab.research.google.com/github/hyang0129/Keras-CoDeepNEAT/blob/master/LAM_CODEEPNEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
c = '''git init .
git remote add -t \* -f origin https://github.com/hyang0129/Keras-CoDeepNEAT
git checkout master
git reset --hard origin/master
'''

import os 
os.system(c)

c = """git pull
git reset --hard origin/master"""
os.system(c)

import sys
sys.path.insert(0,os.getcwd())


In [9]:
!sudo apt-get install graphviz graphviz-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.40.1-2).
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 30 not upgraded.
Need to get 2,120 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-common all 2.24.32-1ubuntu1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-0 amd64 2.24.32-1ubuntu1 [1,769 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgail18 amd64 2

In [10]:
!pip install pygraphviz

     |████████████████████████████████| 122kB 8.5MB/s 
  Created wheel for pygraphviz: filename=pygraphviz-1.7-cp37-cp37m-linux_x86_64.whl size=166115 sha256=b33ee9bbd4df89c69e139430a831ede0f69012c5a4056de02cfe2f689e33946e
  Stored in directory: /root/.cache/pip/wheels/32/59/00/14934a4292c4359eeabcdbf90f33d309b55d0f1be8a1262523
Successfully built pygraphviz


In [3]:
!pip install -r requirements.txt


  Using cached https://files.pythonhosted.org/packages/f8/ba/2d058dcf1b85b9c212cc58264c98a4a7dd92c989b798823cc5690d062bb2/Keras-2.2.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/29/6b4f1e02417c3a1ccc85380f093556ffd0b35dc354078074c5195c8447f2/tensorflow-1.13.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/85/08/f20aef11d4c343b557e5de6b9548761811eb16e438cee3d32b1c66c8566b/networkx-2.3.zip
  Using cached https://files.pythonhosted.org/packages/33/d1/b1479a770f66d962f545c2101630ce1d5592d90cb4f083d38862e93d16d2/pydot-1.4.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/b1/016e657586843f40b4daa66127ce1ee9e3285ff15baf5d80946644a98aeb/graphviz-0.11.1-py2.py3-none-any.whl
     |████████████████████████████████| 6.7MB 7.3MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 368kB 37.9MB/s 
     |████████████████████████████████| 3.2MB 43.4MB/s

In [ ]:
# import sys
# sys.path.append("..")
# sys.path.insert(0,os.getcwd())



In [4]:
import keras, logging, random, pydot, copy, uuid, os, csv, json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from enum import Enum, auto
from typing import List
from keras.utils.vis_utils import plot_model
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from sklearn.preprocessing import scale
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras import regularizers
import imp

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [5]:
kerascodeepneat = imp.load_source("kerascodeepneat", "./base/kerascodeepneat.py")


In [6]:
def run_cifar10_full(generations, training_epochs, population_size, blueprint_population_size, module_population_size, n_blueprint_species, n_module_species, final_model_training_epochs):
    from keras.datasets import cifar10

    #Set parameter tables
    global_configs = {
        "module_range" : ([1, 3], 'int'),
        "component_range" : ([1, 3], 'int')
    }
    input_configs = {
    "module_range" : ([1, 1], 'int'),
    "component_range" : ([1, 1], 'int')
    }
    output_configs = {
        "module_range" : ([1, 1], 'int'),
        "component_range" : ([1, 1], 'int')
    }

    possible_components = {
        "conv2d": (keras.layers.Conv2D, {"filters": ([16,48], 'int'), "kernel_size": ([1, 3, 5], 'list'), "strides": ([1], 'list'), "data_format": (['channels_last'], 'list'), "padding": (['same'], 'list'), "activation": (["relu"], 'list')}),
        #"dense": (keras.layers.Dense, {"units": ([8, 48], 'int')})
    }
    possible_inputs = {
        "conv2d": (keras.layers.Conv2D, {"filters": ([16,64], 'int'), "kernel_size": ([1], 'list'), "activation": (["relu"], 'list')})
    }
    possible_outputs = {
        "dense": (keras.layers.Dense, {"units": ([32,256], 'int'), "activation": (["relu"], 'list')})
    }

    possible_complementary_components = {
        #"maxpooling2d": (keras.layers.MaxPooling2D, {"pool_size": ([2], 'list')}),
        "dropout": (keras.layers.Dropout, {"rate": ([0, 0.5], 'float')})
    }
    possible_complementary_inputs = None
    possible_complementary_outputs = {
        "dense": (keras.layers.Dense, {"units": ([10,10], 'int'), "activation": (["softmax"], 'list')})
    }

    
    # The data, split between train and test sets:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    num_classes = 10

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    validation_split = 0.15
    #training
    batch_size = 128

    #data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        )
    datagen.fit(x_train)

    my_dataset = kerascodeepneat.Datasets(training=[x_train, y_train], test=[x_test, y_test])
    my_dataset.SAMPLE_SIZE = 20000
    my_dataset.TEST_SAMPLE_SIZE = 2000

    logging.basicConfig(filename='execution.log',
                        filemode='w+', level=logging.INFO,
                        format='%(levelname)s - %(asctime)s: %(message)s')
    logging.addLevelName(21, "TOPOLOGY")

    logging.warning('This will get logged to a file')
    logging.info(f"Hi, this is a test run.")

    compiler = {"loss":"categorical_crossentropy", "optimizer":"keras.optimizers.Adam(lr=0.005)", "metrics":["accuracy"]}

    # Set configurations for full training session (final training)
    es = EarlyStopping(monitor='val_acc', mode='auto', verbose=1, patience=15)
    mc = ModelCheckpoint('best_model_checkpoint.h5', monitor='val_accuracy', mode='auto', verbose=1, save_best_only=True)
    csv_logger = CSVLogger('training.csv')
    custom_fit_args = {"generator": datagen.flow(x_train, y_train, batch_size=batch_size),
    "steps_per_epoch": x_train.shape[0] // batch_size,
    "epochs": training_epochs,
    "verbose": 1,
    "validation_data": (x_test,y_test),
    "callbacks": [es, csv_logger]
    }                        
    improved_dataset = kerascodeepneat.Datasets(training=[x_train, y_train], test=[x_test, y_test])
    improved_dataset.custom_fit_args = custom_fit_args
    my_dataset.custom_fit_args = None

    # Initiate population
    population = kerascodeepneat.Population(my_dataset, input_shape=x_train.shape[1:], population_size=population_size, compiler=compiler)
  
    # Start with random modules
    population.create_module_population(module_population_size, global_configs, possible_components, possible_complementary_components)
    population.create_module_species(n_module_species)

    # Start with random modules
    population.create_blueprint_population(blueprint_population_size,
                                            global_configs, possible_components, possible_complementary_components,
                                            input_configs, possible_inputs, possible_complementary_inputs,
                                            output_configs, possible_outputs, possible_complementary_outputs)
    population.create_blueprint_species(n_blueprint_species)

    # Iterate generating, fitting, scoring, speciating, reproducing and mutating.
    iteration = population.iterate_generations(generations=generations,
                                                training_epochs=training_epochs,
                                                validation_split=validation_split,
                                                mutation_rate=0.5,
                                                crossover_rate=0.2,
                                                elitism_rate=0.1,
                                                possible_components=possible_components,
                                                possible_complementary_components=possible_complementary_components)

    print("Best fitting: (Individual name, Blueprint mark, Scores[test loss, test acc], History).\n", (iteration))

    # Return the best model
    best_model = population.return_best_individual()

    # Set data augmentation for full training
    population.datasets = improved_dataset
    print("Using data augmentation.")

    try:
        print(f"Best fitting model chosen for retraining: {best_model.name}")
        population.train_full_model(best_model, final_model_training_epochs, validation_split, custom_fit_args)
    except:
        population.individuals.remove(best_model)
        best_model = population.return_best_individual()
        print(f"Best fitting model chosen for retraining: {best_model.name}")
        population.train_full_model(best_model, final_model_training_epochs, validation_split, custom_fit_args)
  
  
def create_dir(dir):
    if not os.path.exists(os.path.dirname(dir)):
        try:
            os.makedirs(os.path.dirname(dir))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [ ]:
generations = 2
training_epochs = 2
final_model_training_epochs = 2
population_size = 1
blueprint_population_size = 10
module_population_size = 30
n_blueprint_species = 3
n_module_species = 3


create_dir("models/")
create_dir("images/")

run_cifar10_full(generations, training_epochs, population_size, blueprint_population_size, module_population_size, n_blueprint_species, n_module_species, final_model_training_epochs)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Generating 1 components
Generating 3 components


/usr/local/lib/python3.7/dist-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
/usr/local/lib/python3.7/dist-packages/networkx/drawing/nx_pylab.py:676: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if cb.iterable(node_size):  # many node sizes


Generating 3 components
Generating 1 components
Generating 1 components
Generating 3 components
Generating 2 components
Generating 2 components
Generating 3 components
Generating 2 components
Generating 2 components
Generating 1 components
Generating 2 components
Generating 2 components
Generating 3 components
Generating 3 components
Generating 2 components
Generating 2 components
Generating 2 components
Generating 2 components
Generating 1 components
Generating 1 components
Generating 1 components
Generating 3 components
Generating 2 components
Generating 2 components
Generating 1 components
Generating 2 components
Generating 2 components
Generating 3 components
Generating 1 modules
Generating 1 components
Generating 1 components


/usr/local/lib/python3.7/dist-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
/usr/local/lib/python3.7/dist-packages/networkx/drawing/nx_pylab.py:676: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if cb.iterable(node_size):  # many node sizes


Generating 3 modules
Generating 1 components
Generating 1 components
Generating 3 modules
Generating 1 components
Generating 1 components
Generating 1 modules
Generating 1 components
Generating 1 components
Generating 3 modules
Generating 1 components
Generating 1 components
Generating 1 modules
Generating 1 components
Generating 1 components
Generating 3 modules
Generating 1 components
Generating 1 components
Generating 3 modules
Generating 1 components
Generating 1 components
Generating 2 modules
Generating 1 components
Generating 1 components
Generating 1 modules
Generating 1 components
Generating 1 components
 -- Iterating generation 0 -- 
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/dist-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
/usr/local/lib/python3.7/dist-packages/networkx/drawing/nx_pylab.py:676: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if cb.iterable(node_size):  # many node sizes


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 14839 samples, validate on 2619 samples
Epoch 1/2
11136/14839 [=====================>........] - ETA: 1:15 - loss: 2.3374 - acc: 0.1000